# North coast kelp
## Prep for DataONE
Submission guidelines: https://opc.dataone.org/support

In [2]:
## Imports

import pandas as pd
import numpy as np
import csv, pyodbc
import pickle
import datetime

In [3]:
## Ensure my general functions for the MPA data integration project can be imported, and import them

import sys
sys.path.insert(0, '/Users/dianalg/PycharmProjects/PythonScripts/MPA data integration/')

import WoRMS # functions for querying WoRMS REST API

## Connect to db and retrieve data

Note that (as described in the Wiki) the original database filename did not work because it included underscores. I renamed 'Abalone_DiveSurveys_EH_06242020.mdb' to 'AbaloneDiveSurveys-06242020.mdb' to fix this problem.

Also note that **Microsoft does not produce Access OBDC drivers for mac. So now that I'm on a mac, I won't be able to access the actual database without [workarounds](https://github.com/mkleehammer/pyodbc/wiki/Connecting-to-Microsoft-Access).** Fortunately, I've already extracted and saved the data. So...

## Load data

In [8]:
## Function to load data

def load_table(tbl_name):
    """Takes tbl_name (a string) and loads saved data from that table."""
    
    # Get filenames
    col_name = tbl_name + '_cols.data'
    data_name = tbl_name + '.csv'
    
    # Retrieve column names
    with open(col_name, 'rb') as file:
        cols = pickle.load(file)
        
    # Load data
    data = pd.read_csv(data_name, header=None, names=cols)
    return(data)

In [9]:
## Load data

site = load_table('tblSite')
survey = load_table('tblSurvey')
count = load_table('tblCounts')
species = load_table('tblSpecies')
size = load_table('Tbl_New_size')
substrate = load_table('tblSubstrate')
habitat = load_table('tblHabitat')

The data tables are as follows:
- **site**: Contains the site name (SITE) and it's associated two or three letter SiteID
- **survey**: Contains the Survey_ID, SiteID, a unique Survey_Num, a description of the type of survey (SURVEY), the date of the survey (DATE), information about depth (Avg Depth, Min_DEPTH, Max_Depth), information about location (SLAT, SLONG, ELAT, ELONG, SLAT_old, SLONG_old, ELAT_old, ELONG_old, SLAT_DD, SLONG_DD, ELAT_DD, ELONG_DD), and comments (COMMENTS).
- **count**: Contains the Survey_Num, a Layer/Quadrat value indicating the 5 m block along the transect surveyed and whether it was on the left (L) or right (R), the SpeciesID, and the number observed (COUNT)
- **species**: Contains the SpeciesID, common name (SPECIES), scientific name (Scientific) and Notes.
- **size**: Contains sizes (SIZE) of ~ 30 or fewer individuals of target species (SpeciesID) obtained during a survey (Survey_Num).
- **substrate**: Contains the percentage (%Total) of each biotic and abiotic substrate type (HabitatID) for a given Survey_Num. A Subsample column seems to indicate whether the observation was associated with the left (L) side of the transect, the right (R) side of the transect, or both (LR). **Laura said these measurements are taken at the 0, 10, 20 and 30 m marks. Have the values been averaged here? Addded?**
- **habitat**: Contains substrate type codes (HabitatID) and descriptions (HABITAT).

For DataONE, I suggest creating the following tables (based on DataONE guidelines):
- **Site** table, containing site codes, site names, coordinates, CA_MPA_Name_Short, and LTM_project_short_code
- **Species** table, containing species codes, scientific name, ideally common name as well, major taxonomic ranks, WoRMS ID, and species_definition
- **Count** table, containing the number of each organism observed in each layer of each transect during each survey
- **Percent cover** table, containing the percentage of each biotic and abiotic substrate type observed on each transect of each survey
- **Size** table, containing the sizes of organisms sampled during each survey

First, I'm going to tidy the survey table by removing survey types that Laura doesn't want included. I might also limit the columns to those that seem relevant to me, although **Laura should weigh in on whether some of these should remain**. Then I'll work through problems with each of these proposed tables.

In [75]:
## Tidy survey table

# Select relevant columns
sur = survey[[
    'Survey_ID', 
    'SiteID', 
    'Survey_Num', 
    'SURVEY', 
    'DATE', 
    'Avg Depth',
    'Min_DEPTH', 
    'Max_Depth',
    'SLAT',
    'SLONG',
    'ELAT',
    'ELONG',
    'SLAT_old',
    'SLONG_old',
    'ELAT_old',
    'ELONG_old',
    'SLAT_DD',
    'SLONG_DD',
    'ELAT_DD',
    'ELONG_DD',
    'COMMENTS',
]]

# Filter survey type as instructed by Laura
print(sur.shape)
surveys_to_keep = [
    'Transect - 30m (Rapid Emergent)',
    'Transect - 30mx2m (Emergent)',
    'Transect - 30m (Emergent)',
]
sur = sur[sur['SURVEY'].isin(surveys_to_keep)]
print(sur.shape)

# View
sur.head()

(3904, 21)
(2943, 21)


,Survey_ID,SiteID,Survey_Num,SURVEY,DATE,Avg Depth,Min_DEPTH,Max_Depth,SLAT,SLONG,...,ELONG,SLAT_old,SLONG_old,ELAT_old,ELONG_old,SLAT_DD,SLONG_DD,ELAT_DD,ELONG_DD,COMMENTS
0,4790,FR,FR18-D5-1,Transect - 30m (Rapid Emergent),2018-07-12 00:00:00,NaN,48.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,RAPID EMERGENT SURVEY. D5: 1 OUT OF 2 TRANSECT...
1,4919,ALB,ALB18-A02-1,Transect - 30m (Rapid Emergent),2018-08-30 00:00:00,NaN,7.0,10.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,RAPID EMERGENT SITE A02: 1 OF 2 TRANSECTS. HEA...
2,4920,ALB,ALB18-A02-2,Transect - 30m (Rapid Emergent),2018-08-30 00:00:00,NaN,10.0,14.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,RAPID EMERGENT SITE A02: 2 OF 2 TRANSECTS. HEA...
3,4915,ALB,ALB18-A1-1,Transect - 30m (Rapid Emergent),2018-08-30 00:00:00,NaN,12.0,16.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,RAPID EMERGENT SITE A1: 1 OF 4 TRANSECTS. HEAD...
4,4916,ALB,ALB18-A1-2,Transect - 30m (Rapid Emergent),2018-08-30 00:00:00,NaN,8.0,8.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,RAPID EMERGENT SITE A1: 2 OF 4 TRANSECTS. HEAD...


## Site table

In [57]:
## Remove sites that have no survey data

no_surveys = []
for s in site['SiteID'].unique():
    if s not in sur['SiteID'].unique():
        no_surveys.append(s)

site_clean = site[~site['SiteID'].isin(no_surveys)].copy()

In [58]:
## Change column names to something sensible

site_clean.columns = ['SiteID', 'SiteName']

In [59]:
## Add missing information

site_clean['Lat'] = np.nan
site_clean['Lon'] = np.nan
site_clean['CA_MPA_Name_Short'] = ''
site_clean['LTM_project_short_code'] = ''
site_clean

,SiteID,SiteName,Lat,Lon,CA_MPA_Name_Short,LTM_project_short_code
0,ALB,Albion Bay,NaN,NaN,,
1,BR,Bodega Marine Life Refuge (BML),NaN,NaN,,
3,CC,Caspar Cove,NaN,NaN,,
4,FM,Fisk Mill Cove,NaN,NaN,,
6,FR,Fort Ross State Park,NaN,NaN,,
7,HMS,Hopkins Marine Station,NaN,NaN,,
11,MC,Moat Creek,NaN,NaN,,
14,OC,Ocean Cove,NaN,NaN,,
15,PA,Point Arena,NaN,NaN,,
16,PC,Point Cabrillo Lighthouse Reserve,NaN,NaN,,


**Problems:**
- There are a bunch of sites in the site table that do not appear in the survey table. **Remove these?**
- Sites need to be matched to the appropriate CA_MPA_Name_Short value. This column would be left blank if the site is not inside an MPA (e.g. is a reference site)
- Which LTM_project_short_code to use? LTM_Kelp_SRock? (Or NA if site is not part of long term MPA monitoring)
- Correct lat, lon for each site in WGS84 decimal degrees needs to be provided. Coordinates given previously were wonky (see Site location information.png)

## Survey

In [157]:
## Clean Survey_Num - this is probably the best ID column (unique, no missing values)

# Choose ID row - probably Survey_Num is best (unique, no missing values)
sur_num_clean = sur['Survey_Num'].copy()

# Clean leading or lagging whitespace
sur_num_clean = sur_num_clean.str.strip()

# Identify survey numbers that do not fit the formula
do_not_fit = sur_num_clean[~sur_num_clean.str.fullmatch('[A-Z]{2,3}\d\d-[ABCD]{1,2}\d{1,2}')].to_list()

# Identify survey numbers that do fit
fit = sur_num_clean[sur_num_clean.str.fullmatch('[A-Z]{2,3}\d\d-[ABCD]{1,2}\d{1,2}')].to_list()

In [158]:
## Clean SURVEY

survey_clean = pd.DataFrame({
    'Survey_Num':sur_num_clean,
    'SURVEY':sur['SURVEY']
})

survey_clean['SURVEY'] = 'Transect - 30 m x 2 m (Emergent)'

In [159]:
## Clean DATE

# There are two surveys that don't have a date. Dropping these for now.
survey_clean['DATE'] = sur['DATE'].copy()
print(survey_clean.shape)
survey_clean.dropna(inplace=True)
print(survey_clean.shape)

# Turn DATE into datetime
survey_clean['DATE'] = pd.to_datetime(survey_clean['DATE'])

# Add year, month, day as required by DataONE
survey_clean['Year'] = survey_clean['DATE'].dt.year
survey_clean['Month'] = survey_clean['DATE'].dt.month
survey_clean['Day'] = survey_clean['DATE'].dt.day

(2943, 3)
(2941, 3)


In [165]:
## Add depth

survey_clean['Min_DEPTH'] = sur.loc[sur['DATE'].isna() == False, 'Min_DEPTH'].copy()
survey_clean['Max_Depth'] = sur.loc[sur['DATE'].isna() == False, 'Max_Depth'].copy()

In [168]:
## Deal with lat, lon

,Survey_ID,SiteID,Survey_Num,SURVEY,DATE,Avg Depth,Min_DEPTH,Max_Depth,SLAT,SLONG,...,ELONG,SLAT_old,SLONG_old,ELAT_old,ELONG_old,SLAT_DD,SLONG_DD,ELAT_DD,ELONG_DD,COMMENTS
0,4790,FR,FR18-D5-1,Transect - 30m (Rapid Emergent),2018-07-12 00:00:00,NaN,48.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,RAPID EMERGENT SURVEY. D5: 1 OUT OF 2 TRANSECT...
1,4919,ALB,ALB18-A02-1,Transect - 30m (Rapid Emergent),2018-08-30 00:00:00,NaN,7.0,10.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,RAPID EMERGENT SITE A02: 1 OF 2 TRANSECTS. HEA...
2,4920,ALB,ALB18-A02-2,Transect - 30m (Rapid Emergent),2018-08-30 00:00:00,NaN,10.0,14.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,RAPID EMERGENT SITE A02: 2 OF 2 TRANSECTS. HEA...
3,4915,ALB,ALB18-A1-1,Transect - 30m (Rapid Emergent),2018-08-30 00:00:00,NaN,12.0,16.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,RAPID EMERGENT SITE A1: 1 OF 4 TRANSECTS. HEAD...
4,4916,ALB,ALB18-A1-2,Transect - 30m (Rapid Emergent),2018-08-30 00:00:00,NaN,8.0,8.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,RAPID EMERGENT SITE A1: 2 OF 4 TRANSECTS. HEAD...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3789,1858,VD,VD99-45,Transect - 30m (Emergent),1999-08-25 00:00:00,0.0,25.0,NaN,391684.0,1234774.0,...,NaN,39 16.84',123 47.74',39 16.84',NaN,NaN,NaN,NaN,NaN,NaN
3790,1859,VD,VD99-46,Transect - 30m (Emergent),1999-08-25 00:00:00,0.0,8.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3791,1860,VD,VD99-47,Transect - 30m (Emergent),1999-08-25 00:00:00,0.0,20.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3792,1861,VD,VD99-48,Transect - 30m (Emergent),1999-08-04 00:00:00,0.0,45.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
(sur.isna().sum()/sur.shape[0])*100

Survey_ID      0.000000
SiteID         0.000000
Survey_Num     0.000000
SURVEY         0.000000
DATE           0.067958
Avg Depth     50.356779
Min_DEPTH      6.761808
Max_Depth     21.644580
SLAT          48.114169
SLONG         48.114169
ELAT          96.024465
ELONG         98.470948
SLAT_old      83.350323
SLONG_old     83.350323
ELAT_old      96.805980
ELONG_old     99.592253
SLAT_DD       29.867482
SLONG_DD      29.867482
ELAT_DD       50.390758
ELONG_DD      50.390758
COMMENTS      16.853551
dtype: float64

**Problems**:
- Do we still need the following columns: TYPE, TIME_of_Day, NUMBER, DIVER (LEFT FOR TRANSECT), Orientation, Buddy (RIGHT FOR TRANSECT), RANGE, TIDEHEIGHT, Format, TIME_MIN, DISTANCE, ArmDescr, SwmDescr, TranDscr, GrwthDescr
    - Percent of rows with missing values for these columns:
        - TYPE (84%)
        - TIME_of_Day (58%)
        - NUMBER (1%) - **This one seems like it might contain something important.**
        - DIVER (LEFT FOR TRANSECT) (13%) - **I can understand wanting to track this information, but do you want it online?**
        - Orientation (100%)
        - Buddy (RIGHT FOR TRANSECT) (21%) - **I can understand wanting to track this information, but do you want it online?**
        - RANGE (98%)
        - TIDEHEIGHT (98%)
        - Format (69%)
        - TIME_MIN (98%)
        - DISTANCE (99%)
        - ArmDescr (100%)
        - SwmDescr (100%)
        - TranDscr (100%)
        - GrwthDescr (100%)
- Survey_Num should be composed of the site code + the last two digits of the year + a letter indicating depth + a transect number. There are numerous exceptions to this formula. I might be able to search for them with regex, but they'll probably need to be corrected by hand. There's also sometimes leading/lagging whitespace. To see Survey_Nums that need to be corrected, look at list `do_not_fix` generated above.
- Suggest using a controlled vocabulary for SURVEY column
- There are two surveys with no date: OC18-B1-4, OC18-B8-1
- Which depth to use? Pref. min and max for DwC conversion. All depths have missing values and will be dropped from an OBIS submission if we get that far.
        
```python
# Calculate % missing rows per column
(survey.isna().sum()/survey.shape[0])*100
```

**Overall problems:**
- How is Laura going to replicate this longer term? My 'clean' tables won't reflect what she has on her Access Db without overhauling the database.